In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 7.4 MB/s 
     |████████████████████████████████| 7.6 MB 28.6 MB/s 
     |████████████████████████████████| 163 kB 56.4 MB/s 


In [3]:
import os

os.chdir("drive/MyDrive/Colab Notebooks/AOI")

In [4]:
import pandas as pd
import os
from PIL import Image
import torch
import torch.nn as nn
import torchvision.models as models
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from tqdm import tqdm
from transformers import get_scheduler
from transformers import ViTFeatureExtractor, ViTModel

# the class for training data
class AOI_Dataset(Dataset):
    def __init__(self, root_dir, annotation_file, feature_extractor=None, transform=None):
        self.root_dir = root_dir
        self.annotations = pd.read_csv(annotation_file)

        # feature extraction
        self.transform = transform
        self.feature_extractor = feature_extractor

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, index):
        img_id = self.annotations.iloc[index, 0]
        img = Image.open(os.path.join(self.root_dir, img_id)).convert("RGB")
        y_label = torch.tensor(float(self.annotations.iloc[index, 1]))
        img = self.transform(img)
        img = self.feature_extractor(img)

        return (img, y_label)

In [5]:
# the structure of the model
class AOI_network(nn.Module):
  def __init__(self, num_classes=6):
    super(AOI_network, self).__init__()
    self.ViT = ViTModel.from_pretrained('google/vit-base-patch16-224')
    self.out_fc = nn.Linear(self.ViT.config.hidden_size, num_classes)
    self.dropout = nn.Dropout(0.1)

  def forward(self, images):
    features = self.ViT(images).pooler_output
    out = self.dropout(self.out_fc(features))
    return out

In [6]:
device = ("cuda" if torch.cuda.is_available() else "cpu")

# transformations can be specified here
# the goal of the second transformation is sharpening the images as the second
# input images for the model
transform_01 = transforms.Compose(
        [
            transforms.Resize((224, 224)),
            transforms.RandomRotation((10)),
            transforms.ToTensor(),
        ]
    )
num_epochs = 5
learning_rate = 2e-4
batch_size = 16
shuffle = True
num_workers = 1

In [7]:
# all the training images must be in the directory "train_images"
model_name_or_path = 'google/vit-base-patch16-224-in21k'
feature_extractor_01 = ViTFeatureExtractor.from_pretrained(model_name_or_path)
dataset = AOI_Dataset("train_images","train.csv", feature_extractor=feature_extractor_01, transform=transform_01)
train_loader = DataLoader(dataset=dataset, shuffle=shuffle, batch_size=batch_size,num_workers=num_workers)

model = AOI_network().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

learning_rate_scheduler = get_scheduler(
        "linear",
        optimizer=optimizer,
        num_warmup_steps=0,
        num_training_steps=num_epochs * len(train_loader),
    )

Downloading:   0%|          | 0.00/160 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/69.7k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/346M [00:00<?, ?B/s]

Some weights of the model checkpoint at google/vit-base-patch16-224 were not used when initializing ViTModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing ViTModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['vit.pooler.dense.bias', 'vit.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
checkpoint = torch.load("checkpoint", map_location=device)
model.load_state_dict(checkpoint['state_dict'])
optimizer.load_state_dict(checkpoint['optimizer'])
epoch = checkpoint['epoch']

In [12]:
print(optimizer)

AdamW (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    eps: 1e-08
    foreach: None
    initial_lr: 0.0002
    lr: 0.0
    maximize: False
    weight_decay: 0.01
)


In [ ]:
for epoch in range(num_epochs):
  model.train()
  loop = tqdm(train_loader, total = len(train_loader), leave = True)
  for imgs, labels in loop:
    imgs = imgs['pixel_values'][0].type(torch.FloatTensor).to(device)
    labels = labels.type(torch.LongTensor).to(device)
    optimizer.zero_grad()
    outputs = model(imgs)
    # print(outputs.shape, labels.shape)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    learning_rate_scheduler.step()
    loop.set_description(f"Epoch [{epoch+1}/{num_epochs}]")
    loop.set_postfix(loss = loss.item())
  checkpoint = {
                'epoch': epoch,
                'state_dict': model.state_dict(),
                'optimizer': optimizer.state_dict()
  }
  torch.save(checkpoint, "checkpoint")

Epoch [5/5]: 100%|██████████| 158/158 [01:25<00:00,  1.84it/s, loss=0.0469]


In [13]:
# an extra class for testing data
class AOI_Dataset_TEST(Dataset):
    def __init__(self, root_dir, annotation_file, feature_extractor=None, transform=None):
        self.root_dir = root_dir
        self.annotations = pd.read_csv(annotation_file)

        # feature extraction
        self.transform = transform
        self.feature_extractor = feature_extractor

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, index):
        img_id = self.annotations.iloc[index, 0]
        img = Image.open(os.path.join(self.root_dir, img_id)).convert("RGB")
        img = self.transform(img)
        img = self.feature_extractor(img)

        return (img)

In [14]:
import numpy as np
model.eval()
# all the testing images must be in the directory "test_images"
transform_02 = transforms.Compose(
        [
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
        ]
    )
dataset_TEST = AOI_Dataset_TEST("test_images", "test.csv", feature_extractor=feature_extractor_01, transform=transform_02)
test_loader = DataLoader(dataset=dataset_TEST, shuffle=False, batch_size=batch_size,num_workers=num_workers)

loop = tqdm(test_loader, total = len(test_loader), leave = True)
pred_labels = []
res = []

for imgs in loop:
  imgs = imgs['pixel_values'][0].type(torch.FloatTensor).to(device)
  outputs = model(imgs)

  pred = torch.argmax(torch.softmax(outputs, dim=1), dim=-1).to(device, dtype=torch.int8)
  pred_labels.append(pred.cpu().detach().numpy().tolist())

# the original csv file "test.csv" containing the names of all images
pred_labels = np.asarray(pred_labels)
for i in pred_labels:
  for j in i:
    res.append(j)
res = np.array(res)
out_df = pd.read_csv("test.csv")
print(res)
print(len(out_df))
out_df["Label"] = res

# the new output file can be specified here
out_df.to_csv("test_output.csv", encoding='utf-8', index=False)

100%|██████████| 634/634 [49:01<00:00,  4.64s/it]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


[1 2 5 ... 1 3 1]
10142
